<H1><요구사항1> titanic_dataset.py분석

In [1]:
import os

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
print("모듈 가져오기")

모듈 가져오기


In [2]:
print("TitanicDataset Class 정의")
class TitanicDataset(Dataset):
    #Dataset 생성자 정의
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X) # feature tensor는 Float타입으로
        self.y = torch.LongTensor(y)  # target tensor는 long타입으로
    
    # feature tensor의 크기 반환
    def __len__(self):
        return len(self.X)
    
    #클래스의 인덱스에 접근시 해당하는 인덱스의 feature와 target을 반환
    def __getitem__(self, idx):
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}
    
    #String 반환
    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}".format(len(self.X), self.X.shape)
        return str

TitanicDataset Class 정의


In [3]:
print("TitanicTestDateset Class 정의")
class TitanicTestDataset(Dataset):
    #생성자 정의   
    def __init__(self, X):
        self.X = torch.FloatTensor(X) # feature tensor는 Float타입으로
        #Test data이므로 Target은 없음
    
    #feature tensor의 크기 반환
    def __len__(self):
        return len(self.X)
    
    #클래스의 인덱스에 접근시 해당하는 인덱스의 feature 반환
    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature}
    
    #String 반환
    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}".format(
        len(self.X), self.X.shape
        )
        return str

TitanicTestDateset Class 정의


In [4]:
print("데이터셋 전치리 함수 정의")
def get_preprocessed_dataset(): #전처리 과정을 전부 하나로 묶어 놓았음
    CURRENT_FIRE_DIR = os.path.dirname(os.path.abspath('./_03_your_code')) #주피터 노트북에서는 __file__에 오류가 있어 임의 변경
    
    #train데이터와 test데이터 path설정 및 csv파일 읽어오기
    train_data_path = os.path.join(CURRENT_FIRE_DIR, "train.csv") 
    test_data_path = os.path.join(CURRENT_FIRE_DIR, "test.csv")   
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    all_df = pd.concat([train_df, test_df], sort=False) #가져온 train데이터와 test데이터 위아래로 합치기(추가적으로 정렬하지 않음)
    
    #전처리 지정된 단계별 진행
    all_df = get_preprocessed_dataset_1(all_df)
    
    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)
    
    #"Survived"값이 null이 아닌 데이터들을 선택해서 Survived열을 제외하고 train_X 생성 
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    #"Survived"열을 선택해 train_y 생성
    train_y = all_df["Survived"]
    #test데이터 feature는 "Survived"값이 null인 데이터들을 선택해서 test_X 생성
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    
    #타이타닉 train dataset 객체 생성
    dataset = TitanicDataset(train_X.values, train_y.values)
    print(dataset)
    trian_dataset, val_dataset = torch.utils.data.random_split(dataset,[0.8,0.2]) # train data와 validation data를 8:2로 나눔(무작위로)
    #타이타닉 test dataset 객체 생성
    test_dataset = TitanicTestDataset(test_X.values)
    print(test_dataset)
    
    return trian_dataset, val_dataset, test_dataset

데이터셋 전치리 함수 정의


In [5]:
print('get_preprocessed_dataset_1 함수 정의')
#Pclass별로 Fare(요금)의 평균값을 계산하여 결측치 보완에 사용
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index() #Pclass 기준으로 그룹화하여 Fare 평균값 계산
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    #Fare_mean을 Pclass 기준으로 기존 데이터와 병합
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    #Fare열에 null이 있으면 해당 PClass 평균 Fare값으로 결측치 채움
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

get_preprocessed_dataset_1 함수 정의


In [6]:
print('get_preprocessed_dataset_2 함수 정의')
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    # 분리할 때 쉼표(,)와 점(.)을 기준으로 분리 
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    #분리된 데이터를 성, 경침, 이름으로 분리
    name_df.columns = ["family_name", "honorific", "name"]
    #각 열의 문자열 앞뒤에 있는 공백을 제거
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    #기존 데이터에 열 기준으로 병합
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

get_preprocessed_dataset_2 함수 정의


In [7]:
print('get_preprocessed_dataset_3 함수 정의')
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    #열 이름을 honorific과 honorific_age_mean으로 설정
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    #기존 데이터에 honorific_age_mean을 honorific기준으로 병합
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    #Age열에 결측치가 있는 경우, honorific_age_mean으로 결측치 처리
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    #불필요한 honorific_age_mean열 삭제
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

get_preprocessed_dataset_3 함수 정의


In [8]:
print('get_preprocessed_dataset_4 함수 정의')
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"] # 함께 탑승한 부모 수와 자식 수 합치기

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

get_preprocessed_dataset_4 함수 정의


In [9]:
print('get_preprocessed_dataset_5 함수 정의')
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ), #위 4가지 경우를 제외하고는 honorific값을 "other"로 변경
    "honorific"
    ] = "other"
    #'Embarked' 열의 결측값을 'missing'으로 채움
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

get_preprocessed_dataset_5 함수 정의


In [10]:
print('get_preprocessed_dataset_6 함수 정의')
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"] #데이터에서 카테고리 변수 선택

    from sklearn.preprocessing import LabelEncoder 
    #LabelEncoder를 사용하여 범주형 변수를 수치형으로 변환
    for category_feature in category_features:
        le = LabelEncoder() #LabelEncoder 객체 생성
        if all_df[category_feature].dtypes == "object": 
            #해당 열의 모든 범주형 값을 LabelEncoder로 학습
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

get_preprocessed_dataset_6 함수 정의


In [96]:
from torch import nn
print('MyModel class 정의')
class MyModel(nn.Module): #nn.Module을 상속받음
    #MyModel class 생성자 정의
    def __init__(self, n_input, n_output):
        super().__init__()
        #nn.Sequential를 사용해 순차적인 신경망 레이어를 정의
        self.model = nn.Sequential(
        nn.Linear(n_input, 30), #n_input에서 30개의 노드로 연결된 선형 레이어
        nn.PReLU(),             #활성화 함수 ReLU적용
        nn.Linear(30, 30),      #30개에서 또 다른 30개의 노드로 연결된 선형 레이어
        nn.PReLU(),            #활성화 함수 ReLU적용
        nn.Linear(30, n_output),#30개의 노드에서 최종 출력 크기인 n_output으로 연결된 선형 레이어
        )
    
    def forward(self, x):
        x = self.model(x) #nn.Sequential로 정의된 모델에 입력 x를 전달 
        return x

MyModel class 정의


In [97]:
print('test함수 정의')
def test(test_data_loader):
    print("[TEST]")
    #데이터 로더에서 첫번째 배치를 가져옴
    batch = next(iter(test_data_loader))
    # 가져온 batch에서 'input' 키에 해당하는 텐서의 크기를 출력
    print('{0}'.format(batch['input'].shape))
    #정의한 MyModel class를 이용하여 MyModel 객체 생성(입력 노드 11개, 출력 노드 2개)
    my_model = MyModel(n_input=11, n_output=2)
    #MyModel객체에 배치의 'input' 데이터를 입력하여 출력값 얻기
    output_batch = my_model(batch['input'])
    #출력값에서 가장 큰값을 가지는 인덱스를 선택하여 예측값 생성
    prediction_batch = torch.argmax(output_batch, dim=1)
    #각 예측값을 892번 부터 시작하는 인덱스와 출력(892번 부터 test.csv파일에 있는 데이터)
    
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item()) #인덱스와 예측값 출력

test함수 정의


In [91]:
#get_preprocessed_dataset()함수 호출해서 데이터 가져오기 및 전처리 진행
train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
#train data, validation data, test data수 출력
print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
))
print("#" * 50, 1)
#train_dataset에 들어 있는 feature값들과 target값을 보여줌
for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

print("#" * 50, 2)
    
#train, validation, test 데이터 로더 만들기 
train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True) #batch크기 16 데이터를 섞어서 로더 생성
validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True) #batch크기 16 데이터를 섞어서 로더 생성
test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset)) #배치 크기는 test_dataset크기와 같이

print("[TRAIN]")
for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

print("[VALIDATION]")
for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

print("#" * 50, 3)

test(test_data_loader)

Data Size: 891, Input Shape: torch.Size([891, 11])
Data Size: 418, Input Shape: torch.Size([418, 11])
train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 2.0000,  0.0000, 27.0000,  1.0000,  0.0000, 21.0000,  2.0000, 21.1792,
         3.0000,  1.0000,  0.0000]): 0
1 - tensor([ 2.0000,  1.0000, 29.0000,  1.0000,  0.0000, 27.7208,  0.0000, 21.1792,
         2.0000,  1.0000,  0.0000]): 0
2 - tensor([ 3.0000,  0.0000,  3.0000,  3.0000,  1.0000, 21.0750,  2.0000, 13.3029,
         1.0000,  4.0000,  0.0000]): 0
3 - tensor([ 1.0000,  1.0000, 23.0000,  0.0000,  1.0000, 63.3583,  0.0000, 87.5090,
         2.0000,  1.0000,  0.0000]): 1
4 - tensor([ 3.0000,  0.0000,  9.0000,  2.0000,  2.0000, 34.3750,  2.0000, 13.3029,
         1.0000,  4.0000,  0.0000]): 0
5 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.2292,  0.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 3.0000,  1.0000, 29.0000,

/var/folders/vt/ywvm73p11mz2cr555rgpk32h0000gn/T/ipykernel_17200/4160793303.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_df["alone"].fillna(0, inplace=True)
/var/folders/vt/ywvm73p11mz2cr555rgpk32h0000gn/T/ipykernel_17200/3044296567.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behav

<h1><요구사항2> titanic 딥러닝 훈련 코드 및 Activation Function 변경해보기

In [14]:
import wandb
import argparse
from datetime import datetime
import torch
from torch import nn, optim
from torch.utils.data import random_split, dataloader

In [15]:
from pathlib import Path
BASE_PATH = str(Path('./').resolve().parent)
print(BASE_PATH)

import sys
sys.path.append(BASE_PATH)

/Users/kimjinseong/git/link_dl


In [83]:
print('get_model_and_optimizer함수 정의')
def get_model_and_optimizer():
    my_model = MyModel(n_input=11, n_output=2)
    #optimizer는 SGD를 이용
    optimizer = optim.Adam(my_model.parameters(), lr=wandb.config.learning_rate)
    return my_model, optimizer

get_model_and_optimizer함수 정의


In [89]:
print('training_loop함수 정의')
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss() #loss함수는 MSELoss함수 사용 
    next_print_epoch = 10
    loss_validation_min = 100
    #epoch만큼 반복
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        #dataLoader에서 1개씩 꺼내기 
        for batch_idx,train_batch in enumerate(train_data_loader):
            #train_batch만 하면은 input에 정확한 값이 들어가지 않아 직접 지정해서 넣어줌
            input, target = train_batch['input'], train_batch['target']
            #model을 돌려서 output을 얻음 
            output_train = model(input)
            #output과 target의 lossfmf 구함 
            loss = loss_fn(output_train, target)
            loss_train += loss.item()
            num_trains += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input, target = validation_batch['input'], validation_batch['target']
                output_validation = model(input)
                loss = loss_fn(output_validation, target)
                loss_validation += loss.item()
                num_validations += 1
        #wandb에 에포크 로깅 
        wandb.log({
            'Epoch': epoch,
            'Training Loss': loss_train / num_trains,
            'validation Loss': loss_validation / num_validations,
        })
        #이부분이 validataion_loss가 기존 최소 validation_loss보다 낮을 때 모델을 저장 
        if loss_validation <= loss_validation_min:
            torch.save(model.state_dict(), 'MyModel.pt')
            loss_validation_min = loss_validation
        #epoch 10번마다 출력을 하기 위해서 다음과 같이 조건문 실행
        if epoch >= next_print_epoch:
            print(
                f'Epoch {epoch},'
                f'Training Loss: {loss_train / num_trains},'
                f'Validation Loss: {loss_validation / num_validations}'
            )
            next_print_epoch += 10

training_loop함수 정의


In [92]:
print('main함수 정의')
def main(epochs, batch_size, learning_rate):
    current_time_str = datetime.now().astimezone().strftime("%Y_%m_%d_%H_%M_%S") #이름을 시간으로 설정하기 위한 변수
    
    #config 선언 (epochs, batch_size, lr, hidden_unit등 정보가 있음)
    config = {
        'epochs': epochs,
        'batch_size': batch_size,
        'learning_rate': learning_rate,
        'n_hidden_unit_list' : [30, 30], # 위에서 히든 레이어가 30, 30으로 설정
    }
    #wandb 생성
    wandb.init(
        mode="online",
        project='Titanic_Model_Training',
        notes='Titanic: Survivor Prediction',
        tags=['my_model', 'Titanic_Survived'],
        name=current_time_str,
        config=config
    )
    print(wandb.config)
    #config값을 이용해서 DataLoader객체를 생성
    train_data_loader =DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    #모델과 옵티마이저를 얻기위해 get_model_and_optimizer함수 호출
    linear_model , optimizer = get_model_and_optimizer()
    training_loop(
        model=linear_model, 
        optimizer=optimizer, 
        train_data_loader=train_data_loader, 
        validation_data_loader=validation_data_loader
    )
    wandb.finish()

main함수 정의


In [98]:
import argparse
#실행문 주피터 노트북에서는 argparse를 사용하기 힘들어서 직접 하이퍼파라미터 값을 집어넣음
wandbT = True
epochs = 100
batch_size = 16
learning_rate = 0.01
main(
    epochs=epochs,
    batch_size=batch_size,
    learning_rate=learning_rate,
)

{'epochs': 100, 'batch_size': 16, 'learning_rate': 0.01, 'n_hidden_unit_list': [30, 30]}
Epoch 10,Training Loss: 0.5021994974878099,Validation Loss: 0.4659024588763714
Epoch 20,Training Loss: 0.45577140152454376,Validation Loss: 0.48417655502756435
Epoch 30,Training Loss: 0.4341097099913491,Validation Loss: 0.43063004066546756
Epoch 40,Training Loss: 0.4225197262234158,Validation Loss: 0.44294166068236035
Epoch 50,Training Loss: 0.4356599062681198,Validation Loss: 0.43457141766945523
Epoch 60,Training Loss: 0.3821828775935703,Validation Loss: 0.4139431118965149
Epoch 70,Training Loss: 0.39096345040533276,Validation Loss: 0.4041881188750267
Epoch 80,Training Loss: 0.39133331692881057,Validation Loss: 0.4076594774572489
Epoch 90,Training Loss: 0.39236524634891085,Validation Loss: 0.5293680566052595
Epoch 100,Training Loss: 0.4332316994667053,Validation Loss: 0.4227656163275242


Epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
Training Loss,▇█▅▆▇▅▅▄▅▃▃▃▅▃▂▂▃▂▄▅▂▂▁▂▂▂▁▁▂▂▁▁▂▃▂▂▁▃▄▃
validation Loss,█▇▄▄▃▄▄▃▃▃▂▃▂▃▂▂▅▄▂▃▃▂▂▄▂▂▄▃▇▁▃▂▅▁▁▄▂▃▃▂
Epoch,100
Training Loss,0.43323
validation Loss,0.42277


다양한 Activation Function을 사용해보았는데 결과가 다음과 같이 나왔다

ReLU: Validation Loss = 0.40931

ELU: Validation Loss = 0.42583

Leaky ReLU: Validation Loss = 0.42914

PReLU: Validation Loss = 0.40539

Tanh: Validation Loss = 0.43259

가장 Validation Loss가 낮게 나온 것은 PReLU이다.

<h1><요구사항3>테스트 및 submission.csv생성

In [99]:
print('test함수에서 test_csv파일 저장까지 하는 기능을 추가하여 정의')
def test_csv(test_data_loader):
    print("[TEST]")
    #데이터 로더에서 첫번째 배치를 가져옴
    batch = next(iter(test_data_loader))
    # 가져온 batch에서 'input' 키에 해당하는 텐서의 크기를 출력
    print('{0}'.format(batch['input'].shape))
    #정의한 MyModel class를 이용하여 MyModel 객체 생성(입력 노드 11개, 출력 노드 2개)
    my_model = MyModel(n_input=11, n_output=2)
    #MyModel객체에 배치의 'input' 데이터를 입력하여 출력값 얻기
    output_batch = my_model(batch['input'])
    #출력값에서 가장 큰값을 가지는 인덱스를 선택하여 예측값 생성
    prediction_batch = torch.argmax(output_batch, dim=1)
    
    #각 예측값을 저장할 리스트
    predictions = []
    
    #각 예측값을 892번 부터 시작하는 인덱스와 출력(892번 부터 test.csv파일에 있는 데이터)
    for idx, prediction in enumerate(prediction_batch, start=892):
        predictions.append({'PassengerId': idx, 'Survived': prediction.item()}) #인덱스와 예측값을 리스트에 저장
        print(idx, prediction.item()) #인덱스와 예측값 출력
    #submission을 만드는 pandas를 이용해서 만듦
    output_csv_path = os.path.join(Path('./').resolve(), 'submission.csv')
    prediction_df = pd.DataFrame(predictions)
    prediction_df.to_csv(output_csv_path, index=False) 
    print('csv저장 완료') #확인 
    

test함수에서 test_csv파일 저장까지 하는 기능을 추가하여 정의


<h4>훈련과정 중 어느 Epoch시점에 테스트를 수행하여 submission.csv를 구성해야하는지 고찰하기

훈련과정 중 보면은 Validation_loss가 올라가는 경우가 발생하는데 그냥 마지막 시점에서 테스트를 수행하면 오히려 성능이 악화될 수도 있기에 일단은 성능은 Validation_loss가 최소가 될때 모델을 저장을 한다. 
training_loop함수에 아래의 코드를 추가로 작성하였다.


In [64]:
#이부분이 validataion_loss가 기존 최소 validation_loss보다 낮을 때 모델을 저장 
#if loss_validation <= loss_validation_min:
#    torch.save(model.state_dict(), 'MyModel.pt')
#    loss_validation_min = loss_validation

In [107]:
test_csv(test_data_loader)

[TEST]
torch.Size([418, 11])
892 1
893 1
894 1
895 1
896 0
897 0
898 1
899 0
900 0
901 0
902 1
903 0
904 0
905 0
906 0
907 0
908 0
909 0
910 1
911 1
912 0
913 0
914 0
915 0
916 0
917 1
918 0
919 1
920 0
921 0
922 0
923 0
924 0
925 0
926 0
927 0
928 0
929 0
930 0
931 0
932 1
933 0
934 1
935 0
936 0
937 1
938 0
939 1
940 0
941 0
942 0
943 0
944 0
945 0
946 0
947 0
948 1
949 0
950 0
951 0
952 0
953 0
954 0
955 0
956 0
957 0
958 0
959 0
960 0
961 0
962 0
963 0
964 1
965 0
966 0
967 0
968 1
969 0
970 0
971 0
972 0
973 0
974 0
975 1
976 0
977 0
978 1
979 0
980 1
981 0
982 0
983 1
984 0
985 1
986 0
987 1
988 0
989 1
990 0
991 1
992 0
993 0
994 1
995 1
996 1
997 0
998 0
999 1
1000 1
1001 0
1002 1
1003 0
1004 0
1005 0
1006 0
1007 0
1008 1
1009 0
1010 0
1011 0
1012 0
1013 1
1014 0
1015 1
1016 1
1017 0
1018 0
1019 0
1020 0
1021 0
1022 1
1023 0
1024 0
1025 1
1026 1
1027 0
1028 1
1029 0
1030 0
1031 0
1032 0
1033 0
1034 0
1035 0
1036 0
1037 0
1038 0
1039 0
1040 0
1041 0
1042 0
1043 1
1044 1
1045 0
1

<H1><요구사항4> submission.csv 제출  등수확인

![test](https://github.com/dallu3123/link_dl_student/blob/main/submission_rank.png?raw=true)

<H1>숙제 후기

이번 과제에서 저는 딥러닝 모델을 처음으로 설계, 훈련시키고 그 결과를 Kaggle에 제출해보았습니다. 모델의 히든 레이어 구성을 [30, 30]으로 설정했지만, 예상한 만큼의 Validation_loss 결과가 나오지 않아 아쉬움을 느꼈습니다. 이로 인해 더 나은 성능을 얻기 위한 히든 레이어의 설정 방법에 대해 고민해보게 되었고, 다양한 구성을 시도하고 싶은 욕구가 생겼습니다. 현재 가장 성능이 좋은 Activation Function은 PReLU로 확인되었는데, 다른 Activation Function도 적용해 보면서 성능을 비교하고자 합니다. 특히, 각 히든 레이어에 서로 다른 Activation Function을 적용할 경우 어떤 결과가 나오는지 실험해보는 것 역시 의미 있을 것이라 기대합니다.

